# PBkcat

## 背景介绍：
催化常数kcat（catalytic number）:是在底物浓度处于饱和状态下，一个酶（或一个酶活性位点）单位时间内转化的底物分子数，其定义了反应的最大化学转化率，值为Vmax/E。

查尔姆斯理工大学（Chalmers University of Technology）的研究团队提出了深度学习方法 DLKcat来预测所有代谢酶与其底物的 kcat值，只需要底物 SMILES 信息和酶的蛋白质序列作为输入，从而为任何物种产生通用的 kcat预测工具。文章中使用pearson系数对模型进行评估（代表预测值与实际值之间的相关性系数），表现还不错，但是在实际测试中该方法预测出来的kcat值误差非常大。

为此，我们希望通过建立PBkcat模型，该模型使用proteinBERT提取蛋白质氨基酸序列特征、使用GNN提取底物分子图特征，再使用attention机制对kcat值进行大规模预测，希望能进一步探究蛋白质和底物分子与kcat值之间的联系，能够更加准确的预测kcat值。

## 相关算法介绍：

### ProteinBERT
2018年，Devlin等提出基于深度双向Transformer的预训练模型ProteinBERT，旨在以一种自然的方式捕获蛋白质的局部和全局表示。ProteinBERT在涵盖不同蛋白质属性（包括蛋白质结构、翻译后修饰和生物物理属性）的多个基准上获得了最先进的性能。

### Substructure-based graph neural network (sub-GNN)
对于底物的处理，我们打算通过GNN来提取底物的分子图特征。分子图以原子为节点，键为边的图形表示，节点存储信息（标签），例如原子类型、电荷、多重性和质量，而边存储键合顺序。每个都可以具有关于芳族和立体异构的信息。

----------

In [ ]:
#@title clone and download dependencies
! git clone https://github.com/950288/PBkcat_test.git
%cd PBkcat_test
! pip install  -r dependencies.txt
! wget -nc -P preprocess ftp://ftp.cs.huji.ac.il/users/nadavb/protein_bert/epoch_92400_sample_23500000.pkl 

In [ ]:
#@title clone and download data
! wget -nc -P data www.95028.top/ML/ML.zip
! unzip -n -d data data/ML.zip

In [ ]:
#@title convert substrate smiles to fingerprints for training
! python preprocess/substrate.py

In [ ]:
#@title preprocess protein
! python preprocess/proteinBERT_global_rep.py

In [ ]:
#@title preprocess protein
! python preprocess/proteinBERT_local_rep.py

In [ ]:
#@title train model
! python model/train_model.py